In [25]:
import xml.etree.ElementTree as ET
from collections import deque

# Charger le fichier XML WOLF
xml_file = r"C:\Users\jokle\Downloads\wolf.xml"
tree = ET.parse(xml_file)
root = tree.getroot()

# Extraire tous les synsets et leurs relations
synset_relations = {}

# Fonction pour ajouter une relation entre synsets
def add_relation(synset_id, related_synset_id):
    if synset_id not in synset_relations:
        synset_relations[synset_id] = set()
    synset_relations[synset_id].add(related_synset_id)

# Extraire les relations
for synset in root.findall(".//SYNSET"):
    synset_id = synset.find("ID").text
    
    # Récupérer les relations (hypernymie, hyponymie, etc.)
    for relation in synset.findall(".//ILR"):
        related_synset_id = relation.text
        add_relation(synset_id, related_synset_id)
        add_relation(related_synset_id, synset_id)  # Ajout symétrique

# Afficher les relations pour un synset donné
print(synset_relations.get('eng-30-00015388-n', []))  # Exemple

def find_shortest_path(word1, word2):
    # Fonction pour trouver le synset d'un mot (ici on suppose que chaque mot appartient à un synset unique)
    def get_synset_for_word(word):
        for synset in root.findall(".//SYNSET"):
            for synonym in synset.findall(".//SYNONYM"):
                for literal in synonym.findall("LITERAL"):
                    if literal.text and literal.text.lower() == word.lower():
                        return synset.find("ID").text
        return None

    synset1 = get_synset_for_word(word1)
    synset2 = get_synset_for_word(word2)
    
    if not synset1 or not synset2:
        return None  # Si un des mots n'est pas trouvé

    # BFS pour trouver le plus court chemin entre synset1 et synset2
    queue = deque([(synset1, 0)])  # (synset_id, distance)
    visited = set()

    while queue:
        current_synset, distance = queue.popleft()

        if current_synset == synset2:
            return distance  # Retourne la distance si synsets égaux

        visited.add(current_synset)

        for neighbor in synset_relations.get(current_synset, []):
            if neighbor not in visited:
                queue.append((neighbor, distance + 1))

    return None  # Aucun chemin trouvé

# Exemple d'utilisation
word1 = "glace"
word2 = "école"
distance = find_shortest_path(word1, word2)
if distance is not None:
    print(f"Le plus court chemin entre '{word1}' et '{word2}' est de longueur {distance}.")
else:
    print(f"Aucun chemin trouvé entre '{word1}' et '{word2}'.")


{'eng-30-00160288-a', 'eng-30-01314388-n', 'eng-30-01313093-n', 'eng-30-01317541-n', 'eng-30-02152740-n', 'eng-30-01321230-n', 'eng-30-00313701-a', 'eng-30-02252634-a', 'eng-30-01318478-n', 'eng-30-01323781-n', 'eng-30-01324610-n', 'eng-30-05216365-n', 'eng-30-01263445-a', 'eng-30-01321456-n', 'eng-30-01324799-n', 'eng-30-01316288-n', 'eng-30-01680756-v', 'eng-30-04474466-n', 'eng-30-00214461-a', 'eng-30-02891444-a', 'eng-30-01315805-n', 'eng-30-05538625-n', 'eng-30-01427010-a', 'eng-30-01958898-a', 'eng-30-01316949-n', 'eng-30-01904698-a', 'eng-30-01324916-n', 'eng-30-01319001-n', 'eng-30-00320284-n', 'eng-30-01488616-a', 'eng-30-00302130-v', 'eng-30-01315330-n', 'eng-30-02656121-a', 'eng-30-14224547-n', 'eng-30-01959111-a', 'eng-30-01617192-v', 'eng-30-01316422-n', 'eng-30-02075612-n', 'eng-30-02152991-n', 'eng-30-01324142-n', 'eng-30-01314910-n', 'eng-30-01324305-n', 'eng-30-04828925-n', 'eng-30-01315581-n', 'eng-30-01320872-n', 'eng-30-01315213-n', 'eng-30-02311060-n', 'eng-30-0019

In [27]:
import xml.etree.ElementTree as ET
import networkx as nx

# Charger le fichier XML
xml_file = r"C:\Users\jokle\Downloads\wolf.xml"
tree = ET.parse(xml_file)  # Mets le bon chemin vers ton fichier
root = tree.getroot()

# Création du graphe
G = nx.Graph()

# Ajouter tous les synsets et leurs relations
for synset in root.findall(".//SYNSET"):
    synset_id = synset.find("ID").text  # Récupère l'ID du synset
    for ilr in synset.findall(".//ILR"):  # Trouve les relations
        related_synset = ilr.text
        G.add_edge(synset_id, related_synset)  # Ajoute une arête entre synsets

# Fonction pour obtenir les synsets d'un mot
def get_synsets_for_word(word):
    synsets = []
    for synset in root.findall(".//SYNSET"):
        for synonym in synset.findall(".//SYNONYM"):
            for literal in synonym.findall("LITERAL"):
                if literal.text and literal.text.lower() == word.lower():
                    synsets.append(synset.find("ID").text)
    return synsets

# Fonction pour récupérer les mots d’un synset
def get_words_from_synset(synset_id):
    words = []
    for synset in root.findall(".//SYNSET"):
        if synset.find("ID") is not None and synset.find("ID").text == synset_id:
            for synonym in synset.findall(".//SYNONYM"):
                for literal in synonym.findall("LITERAL"):
                    words.append(literal.text)
    return words

# Définition des mots à relier
word1 = "économique"
word2 = "maison"

synsets1 = get_synsets_for_word(word1)
synsets2 = get_synsets_for_word(word2)

# Trouver le plus court chemin et afficher le détail
if synsets1 and synsets2:
    try:
        shortest_path = nx.shortest_path(G, source=synsets1[0], target=synsets2[0])
        print(f"📏 Chemin entre '{word1}' et '{word2}':\n")
        
        for synset_id in shortest_path:
            words = get_words_from_synset(synset_id)
            print(f"🔹 Synset {synset_id} → {', '.join(words) if words else 'Aucun mot'}")
        
    except nx.NetworkXNoPath:
        print(f"🚫 Aucun chemin trouvé entre '{word1}' et '{word2}'.")
else:
    print("❌ Un des mots n'existe pas dans WOLF.")


📏 Chemin entre 'économique' et 'maison':

🔹 Synset eng-30-00840212-a → économique
🔹 Synset eng-30-00192613-n → économie, épargne
🔹 Synset eng-30-00037396-n → action
🔹 Synset eng-30-00041188-n → _EMPTY_
🔹 Synset eng-30-02418686-v → jouer
🔹 Synset eng-30-00431893-n → jeu, jeu d'enfant
🔹 Synset eng-30-00432164-n → maison


In [ ]:
import xml.etree.ElementTree as ET

# Charger le fichier XML de WOLF
xml_file = r"C:\Users\jokle\Downloads\wolf.xml"
tree = ET.parse(xml_file)
root = tree.getroot()

def get_synsets_for_word(word):
    synsets = []

    for synset in root.findall(".//SYNSET"):
        synset_id = synset.find("ID").text  # ID du synset
        words_in_synset = [literal.text for literal in synset.findall(".//LITERAL")]

        if word in words_in_synset:
            definition = synset.find("DEF").text if synset.find("DEF") is not None else "Pas de définition"
            synsets.append((synset_id, words_in_synset, definition))

    return synsets

# Test avec un mot
word = "animal"
synsets = get_synsets_for_word(word)

# Afficher les synsets trouvés
for synset_id, words, definition in synsets:
    print(f"🔹 Synset ID: {synset_id}")
    print(f"   Synonymes: {', '.join(words)}")
    print(f"   Définition: {definition}\n")


🔹 Synset ID: eng-30-00015388-n
   Synonymes: animal, bête, créature, faune, être
   Définition: a living organism characterized by voluntary movement

🔹 Synset ID: eng-30-01313093-n
   Synonymes: animal, animalia
   Définition: taxonomic kingdom comprising all living or extinct animals

🔹 Synset ID: eng-30-01778572-a
   Synonymes: animal, bête, sensuel
   Définition: marked by the appetites and passions of the body



In [38]:
G = nx.Graph()  # Rend le graphe non dirigé
import xml.etree.ElementTree as ET

# Charger le fichier XML de WOLF
xml_file = r"C:\Users\jokle\Downloads\wolf.xml"
tree = ET.parse(xml_file)
root = tree.getroot()
def get_synset_for_word(word):
    synsets = []

    # Parcourir tous les synsets
    for synset in root.findall(".//SYNSET"):
        synset_id = synset.find("ID").text  # Récupérer l'ID du synset

        # Chercher dans les synonymes du synset
        for synonym in synset.findall(".//SYNONYM"):
            for literal in synonym.findall("LITERAL"):
                if literal.text and literal.text.lower() == word.lower():
                    synsets.append(synset_id)

    return synsets if synsets else None  # Retourne None si aucun synset trouvé



# Ajouter les relations
for synset in root.findall(".//SYNSET"):
    synset_id = synset.find("ID").text
    for ilr in synset.findall(".//ILR"):
        related_synset = ilr.text
        relation_type = ilr.get("type")

        weight = {
            "synonym": 1,
            "hypernym": 2,
            "hyponym": 2,
            "antonym": 4,
            "be_in_state": 5,
            "also_see": 5
        }.get(relation_type, 3)

        G.add_edge(synset_id, related_synset, weight=weight)


synset1 = get_synset_for_word("chien")
synset2 = get_synset_for_word("chat")

print(f"🔍 Synset de 'chien' : {synset1}")
print(f"🔍 Synset de 'chat' : {synset2}")


if synset1 not in G.nodes():
    print(f"❌ {synset1} n'existe pas dans le graphe")
if synset2 not in G.nodes():
    print(f"❌ {synset2} n'existe pas dans le graphe")

try:
    shortest_path = nx.shortest_path(G, source=synset1, target=synset2, weight="weight")
    print(f"🌟 Plus court chemin entre '{synset1}' et '{synset2}' :")
    print(" ➡ ".join(shortest_path))
    print(f"📏 Longueur du chemin : {len(shortest_path)-1}")  # Nombre d'arêtes parcourues
except nx.NetworkXNoPath:
    print(f"🚫 Aucun chemin trouvé entre '{synset1}' et '{synset2}'")


🔍 Synset de 'chien' : ['eng-30-02084071-n', 'eng-30-02084732-n', 'eng-30-02087551-n', 'eng-30-02710044-n', 'eng-30-03901548-n', 'eng-30-10023039-n', 'eng-30-10114209-n']
🔍 Synset de 'chat' : ['eng-30-00901476-n', 'eng-30-02121234-n', 'eng-30-02121620-n', 'eng-30-02122725-n', 'eng-30-02127808-n', 'eng-30-02985606-n', 'eng-30-09638245-n', 'eng-30-09900153-n', 'eng-30-10153414-n']


TypeError: unhashable type: 'list'